# Pós-processamento Cluster

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 8))
import janitor
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px

import datetime

import sqlite3 as sql

from sklearn.cluster import KMeans
from numpy import unique
from numpy import where


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve


#nomalizing data to its std (x_new= x/std(x))
from scipy.cluster.vq import whiten

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity="all"

In [2]:
!ls ./Novo_02

dbscan.csv  GMM.csv  HC.csv  kmeans.csv  mean_s.csv


In [3]:
dir="/Novo_02/"

In [4]:
df_kmeans=pd.read_csv("."+dir+"kmeans.csv", sep=";", dtype={"CPF":"object"})
df_mean_s=pd.read_csv("."+dir+"mean_s.csv", sep=";", dtype={"CPF":"object"})
df_GMM=pd.read_csv("."+dir+"GMM.csv", sep=";", dtype={"CPF":"object"})
df_dbscan=pd.read_csv("."+dir+"dbscan.csv", sep=";", dtype={"CPF":"object"})
df_HC=pd.read_csv("."+dir+"HC.csv", sep=";", dtype={"CPF":"object"})

# Avaliação com ARI Adjusted Rand Score Index

In [5]:
# adjusted rand score is 1, as the clustering is exactly the same
# df_kmeans, df_mean_s, df_dbscan, df_GMM, df_HC, df_ch

In [6]:
from sklearn.metrics import adjusted_rand_score
target=df_HC
lista=[df_kmeans,df_mean_s,df_dbscan,df_GMM,df_HC]
ari_score={}
for mod1 in lista:
    reg={}
    for mod2 in lista:
        ari=adjusted_rand_score(mod1["cluster"].values, mod2["cluster"].values)
        reg[mod2]=ari
    ari_score[mod1]=reg
    print(mod1)
    print( reg)


TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

In [11]:
mod1.name="teste"
mod1.name()


TypeError: 'str' object is not callable

## EDA

In [ ]:
#Dados de parlamentares
con = sql.connect("../../dados/sql/base_completa.db")
df=pd.read_sql("select * from kpi_cadastro_basico",con)
con.close()

In [ ]:
df_cluster=df_kmeans[["CPF","cluster"]].copy()
df_cluster.info()

In [ ]:
df=pd.merge(df, df_cluster, how="left", on="CPF")
df.info()

## EDA Básico

In [ ]:
df.drop(columns=["201901", '201902','201903',
                 "201904", '201905', '201906', '201907', '201908', '201909',
                 '201910', '201911', '201912', '202001', '202002', '202003',
                 '202004','202005', '202006', '202007', '202008', '202009'], inplace=True)

In [ ]:
df.shape
df.info()
df.sample(10)

In [ ]:
df.info()
df.describe().round(2)

In [ ]:
df.describe(include=np.object)

In [ ]:
# Histogramas "somados" stacked
fig = px.histogram(df,x='ORGAO_TOTAL',color="cluster")
fig.show()

In [ ]:
sns.countplot(x="ORGAO_TOTAL",hue="cluster", data=df)

In [ ]:
variaveis= list(df.select_dtypes(include=[np.number]).columns)
#variaveis=list(df.columns)
target=["cluster"]
variaveis
target

In [ ]:
len(variaveis)//2

In [ ]:
df.columns

In [ ]:
correlação=df.corr()
correlação

In [ ]:
correlação.cluster.sort_values(ascending=False)

In [ ]:
plt.rc('figure', figsize=(16, 16))
sns.heatmap(correlação,annot=True, vmin=-1, vmax=1)

In [ ]:
variaveis
target=["cluster"]

In [ ]:
#preditores=list(set(variaveis)-set(target))
preditores=variaveis
variaveis=target+preditores
correlação=df[variaveis].corr()
correlação

In [ ]:
plt.rc('figure', figsize=(12, 8))
sns.heatmap(correlação,annot=True, vmin=-1, vmax=1)

In [ ]:
#sns.pairplot(df[variaveis], kind="scatter", hue=target[0])
#sns.pairplot(df[variaveis], kind="scatter")

In [ ]:
sns.pairplot(df, y_vars=target, x_vars=preditores, hue=target[0])

## Fatores

In [ ]:
df.info()
unicos=df.nunique()
unicos

In [ ]:
# Tentar as que tem até 47
unicas=5
variaveis_categoricas=[]
for variavel in unicos.index:
    quant=unicos.loc[variavel]
    if quant <= unicas:
        variaveis_categoricas.append(variavel)
        print("Variavel {}, {} ocorrências únicas".format(variavel,quant))
        print(df[variavel].sort_values().unique())

In [ ]:
#left, sales, salary, promotion_last_5years, work_accident

variaveis_categoricas

In [ ]:
variaveis_categoricas.remove("ANO_ELEICAO")
variaveis_categoricas.remove("DS_CARGO")
variaveis_categoricas


In [ ]:
df[variaveis_categoricas].info()

In [ ]:
for variavel in variaveis_categoricas:
    df[variavel]=df[variavel].astype("category")
df.info()


In [ ]:
df.drop(columns=["ANO_ELEICAO", "SQ_CANDIDATO","IDADE","ID_CAMARA","LEG_INICIAL"], inplace=True)

In [ ]:
exibe=['NM_PUBLICO','SG_PARTIDO', 'SG_UE','followers_count', "ORGAO_TOTAL",
           'ORGAO_PARTICIPANTE','ORGAO_GESTOR', 'PERC_PRESENCA', 'GASTO_GABINETE','cluster']
cluster=0
sort="ORGAO_TOTAL"

In [ ]:
df_1=df[exibe][df["cluster"]==cluster]


In [ ]:
x=df_1.groupby("SG_PARTIDO")["NM_PUBLICO"].value_counts()
df_contagem=x.unstack().fillna(0)
df_contagem["TOTAL"]=df_contagem.sum(axis=1)
df_contagem.sort_values("TOTAL", ascending=False, inplace=True)
df_contagem=df_contagem[["TOTAL"]].copy()
df_1.shape
df_contagem=df_contagem.reset_index(drop=False)
fig = px.treemap(df_contagem, path=['SG_PARTIDO'], values='TOTAL'
                  )
fig.show()

In [ ]:
x=df.groupby(["cluster","SG_PARTIDO"],as_index=False)["SG_PARTIDO"].sum()
x.head()
#df_contagem=x.unstack().fillna(0)
#df_contagem.head()
#df_contagem["TOTAL"]=df_contagem.sum(axis=1)
#df_contagem.sort_values("TOTAL", ascending=False, inplace=True)
#df_contagem=df_contagem[["TOTAL","SG_PARTIDO"]].copy()
#df.shape
#df_contagem.head()
#df_contagem=df_contagem.reset_index(drop=False)

#fig = px.treemap(df_contagem, path=['cluster',"SG_PARTIDO"], values='TOTAL')
#fig.show()

In [ ]:
variaveis_numericas=list(df.select_dtypes(include=[np.number]).columns)
variaveis_numericas

### Histogramas

In [ ]:
for variavel in variaveis_categoricas:
    df[variavel].value_counts().plot(kind="bar")
    plt.xlabel(variavel)
    plt.ylabel("Contagem")
    plt.title(variavel)
    plt.show()

In [ ]:
df.columns

In [ ]:
exibe=['NM_PUBLICO','SG_PARTIDO', 'SG_UE','followers_count', "ORGAO_TOTAL",
           'ORGAO_PARTICIPANTE','ORGAO_GESTOR', 'PERC_PRESENCA', 'GASTO_GABINETE','cluster']
cluster=1
sort="ORGAO_TOTAL"

In [ ]:
df[exibe][df["cluster"]==cluster].shape
df[exibe][df["cluster"]==cluster].sort_values(sort, ascending=False)


### Boxplots - Identificando, verificando e tratando outliers


In [ ]:
for cat in variaveis_categoricas:
    for num in variaveis_numericas:
        sns.boxplot(x=cat, y=num, data=df)
        plt.xlabel(cat)
        plt.ylabel(num)
        plt.title(cat)
        plt.show()


In [ ]:
df

## Criação das variáveis dummies

In [ ]:
df.columns

In [ ]:
variaveis_categoricas

In [ ]:
variaveis_categoricas.remove(target[0])

In [ ]:
#variaveis_categoricas.remove(target[0])
#variaveis_dummy.remove(target[0])
variaveis_categoricas
variaveis_dummy=[]

In [ ]:
# Trata variaveis binárias (0-1)
for variavel in variaveis_dummy:
    df=pd.concat([df, pd.get_dummies(df[variavel].astype(object), prefix=variavel, drop_first=True)], axis=1)
    df.drop(columns=[variavel], inplace=True)
df.sample(5)
df.columns

In [ ]:
#Trata variaveis categóricas
variaveis_categoricas=list(set(variaveis_categoricas)-set(variaveis_dummy))
for variavel in variaveis_categoricas:
    df=pd.concat([df, pd.get_dummies(df[variavel].astype(object), prefix=variavel)], axis=1)
    df.drop(columns=[variavel], inplace=True)
df.sample(5)
df.columns

In [ ]:
preditores=list(df.select_dtypes(include=[np.number]).columns)
target
preditores=list(set(preditores)-set(target))
preditores
target

## Escala dos preditores

In [ ]:
df.info()
df=df.fillna(0)


In [ ]:
def verifica_escala(preditores,df):
    x=df[preditores].values
    plt.plot(x.min(axis=0),"o", label="min")
    plt.plot(x.max(axis=0),"^", label="max")
    plt.legend(loc="best")
    plt.xlabel("Feature index")
    plt.ylabel("Feature magnitude")
    plt.yscale("log")
    print(df[preditores].max().round(3))

In [ ]:
verifica_escala(preditores, df)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df[preditores].values)
x=scaler.transform(df[preditores].values)
df_scaled=pd.DataFrame(x,columns=preditores)
df_scaled[target]=df[target]
df_scaled.head()
df_original=df.copy()
df=df_scaled.copy()

In [ ]:
target

## treino e teste


In [ ]:
df_treino, df_teste= train_test_split(df, test_size=0.3, random_state=42, stratify=df[target])

In [ ]:
df_treino.shape
df_teste.shape

## Função para classificação de algoritimos

In [ ]:
def classificação(modelo,target,preditores,treino,teste):
    x_treino=treino[preditores].values
    y_treino=treino[target].values
    y_treino=y_treino.ravel()
    
    x_teste=teste[preditores].values
    y_teste=teste[target].values
    
    modelo.fit(x_treino, y_treino)
    y_treino_pred=modelo.predict(x_treino)
    y_teste_pred=modelo.predict(x_teste)
    print(modelo)
    plot_confusion_matrix(modelo, x_teste,y_teste, cmap='Blues')
    print("Acurácia no treino: {:.3f}".format(modelo.score(x_treino, y_treino)))
    print("Acuracia no teste: {:.3f}".format(modelo.score(x_teste, y_teste)))
    print("Precisão no teste: {:.3f}".format(precision_score(y_teste, y_teste_pred)))
    print("Recall no teste: {:.3f}".format(recall_score(y_teste, y_teste_pred)))
    print("F1 no teste: {:.3f}".format(f1_score(y_teste, y_teste_pred)))
    plot_roc_curve(modelo, x_teste, y_teste)
    mod={}
    mod["modelo"]=modelo
    mod["acuracia_teste"]=modelo.score(x_teste, y_teste)
    mod["acuracia_treino"]=modelo.score(x_treino, y_treino)
    return mod

## Árvore de Decisão 

## Verificando a profundidade da árvore e a acurácia

In [ ]:
x_treino=df_treino[preditores].values
y_treino=df_treino[target].values
y_treino=y_treino.ravel()
x_teste=df_teste[preditores].values
y_teste=df_teste[target].values

valores=range(2,50)

ac_treino=[]
ac_teste=[]
    
for val in valores:
    modelo= DecisionTreeClassifier(max_depth=val)
    modelo.fit(x_treino, y_treino)
    #y_treino_pred=modelo.predict(x_treino)
    #y_teste_pred=modelo.predict(x_teste)
    ac_treino.append(modelo.score(x_treino, y_treino))
    ac_teste.append(modelo.score(x_teste, y_teste))
    
df_acuracia=pd.DataFrame({"Parametro":valores,"acuracia_treino":ac_treino,"acuracia_teste":ac_teste }, index=valores) 
df_acuracia[["acuracia_treino","acuracia_teste"]].plot()
df_acuracia.sort_values("acuracia_teste",inplace=True, ascending=False)
df_acuracia

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
modelo = DecisionTreeClassifier(max_depth=27)

result=classificação(modelo,target, preditores, df_treino, df_teste)
modelo=result["modelo"]
lista_modelos.append(result)

### Visualizando a árvore

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plot_tree(modelo, feature_names=preditores, filled=True,
          class_names=['Fica', 'Sai']);

In [ ]:
modelo.feature_importances_
preditores
importancia=pd.Series(modelo.feature_importances_, index=preditores)
importancia=importancia.sort_values(ascending=True)
importancia.plot(kind="barh")

## SVM

## Verificando o valor de C

In [ ]:
x_treino=df_treino[preditores].values
y_treino=df_treino[target].values
y_treino=y_treino.ravel()
x_teste=df_teste[preditores].values
y_teste=df_teste[target].values

valores=range(1,35)

ac_treino=[]
ac_teste=[]
    
for val in valores:
    modelo=  SVC(C=val)
    modelo.fit(x_treino, y_treino)
    #y_treino_pred=modelo.predict(x_treino)
    #y_teste_pred=modelo.predict(x_teste)
    ac_treino.append(modelo.score(x_treino, y_treino))
    ac_teste.append(modelo.score(x_teste, y_teste))
    
df_acuracia=pd.DataFrame({"Parametro":valores,"acuracia_treino":ac_treino,"acuracia_teste":ac_teste }, index=valores) 
df_acuracia[["acuracia_treino","acuracia_teste"]].plot()
df_acuracia

## Verificando o valor de gamma

In [ ]:
x_treino=df_treino[preditores].values
y_treino=df_treino[target].values
y_treino=y_treino.ravel()
x_teste=df_teste[preditores].values
y_teste=df_teste[target].values

valores=list(np.arange(0.1,5,0.5))

ac_treino=[]
ac_teste=[]
    
for val in valores:
    modelo=  SVC(C=30, gamma=val)
    modelo.fit(x_treino, y_treino)
    #y_treino_pred=modelo.predict(x_treino)
    #y_teste_pred=modelo.predict(x_teste)
    ac_treino.append(modelo.score(x_treino, y_treino))
    ac_teste.append(modelo.score(x_teste, y_teste))
    
df_acuracia=pd.DataFrame({"Parametro":valores,"acuracia_treino":ac_treino,"acuracia_teste":ac_teste }, index=valores) 
df_acuracia[["acuracia_treino","acuracia_teste"]].plot()
df_acuracia

In [ ]:

modelo = SVC(C=30, gamma=4.1)


result=classificação(modelo,target, preditores, df_treino, df_teste)
modelo=result["modelo"]
lista_modelos.append(result)


# Comparando todos os modelos

In [ ]:
lista_modelos

In [ ]:
def compara_modelos(lista,target,preditores, teste):
    x=df[preditores].values
    y=df[target].values
    ax = plt.gca()
    for i in lista:
        modelo=i["modelo"]
        plot_roc_curve(modelo, x, y, ax=ax)

In [ ]:
compara_modelos(lista_modelos, target, preditores, df_teste)

# Aplicando balanceamento

In [ ]:
preditores=list(df.columns)
target
preditores=list(set(preditores)-set(target))
preditores
target

In [ ]:
#from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_treino=df_treino[preditores].values
y_treino=(df_treino[target].values).ravel()
y_treino.shape
X_treino_bal, y_treino_bal = smote.fit_sample(X_treino, y_treino)
X_treino_bal.shape
y_treino_bal.shape
y_treino_bal=y_treino_bal.reshape((-1,1))
y_treino_bal.shape

variaveis=preditores+target

df_treino_bal=pd.DataFrame(X_treino_bal, columns=preditores)
df_treino_bal[target[0]]=y_treino_bal
df_treino_bal.sample(5)

In [ ]:
# balanceado
X_treino=df_treino_bal[preditores].values
y_treino=(df_treino_bal[target].values).ravel()
X_treino.shape
y_treino.shape
X_teste=df_teste[preditores].values
y_teste=np.array(df_teste[target]).ravel()
y_treino[1:10]

In [ ]:
rlog_bal = LogisticRegression(solver = 'sag', max_iter = 10000).fit(X_treino, y_treino)


In [ ]:
rlog_bal.intercept_
rlog_bal.coef_.round(3)

In [ ]:
y_proba= rlog_bal.predict_proba(X_treino).round(3)
y_proba[1:10]

In [ ]:
y_treino_pred=rlog_bal.predict(X_treino)
y_treino_pred[1:10]

In [ ]:
y_teste_rlog=rlog_bal.predict(X_teste)
y_teste_rlog[1:10]

In [ ]:
rlog_bal.score(X_treino, y_treino)

In [ ]:
rlog_bal.score(X_teste, y_teste)

## Matriz de confusão - scklearn

In [ ]:
#confusion_matrix(y, model.predict(x))
confusion_matrix(y_treino,y_treino_pred)


In [ ]:
plot_confusion_matrix(rlog_bal, X_treino, y_treino, cmap='Blues')

In [ ]:
plot_confusion_matrix(rlog_bal, X_teste, y_teste, cmap='Blues')

In [ ]:
#Do sckit 
#- Acurácia: % de predições corretas
#- Precisão: % de predições corretas dentro da classe positiva
#- Recall: % de predições corretas dentro da classe esperada como positiva
print(classification_report(y_treino, y_treino_pred, target_names=['No', 'Yes']))

In [ ]:
#Do sckit 
#- Acurácia: % de predições corretas
#- Precisão: % de predições corretas dentro da classe positiva
#- Recall: % de predições corretas dentro da classe esperada como positiva
print(classification_report(y_teste, y_teste_rlog, target_names=['No', 'Yes']))
print(f'Acurácia:{accuracy_score(y_teste, y_teste_rlog)}\n\
Precisão:{precision_score(y_teste, y_teste_rlog)}\n\
Recall:{recall_score(y_teste, y_teste_rlog)}')


## Curva ROC e AUC

In [ ]:
plot_roc_curve(rlog_bal, X_teste, y_teste)

## KNN

In [ ]:
knn_bal = KNeighborsClassifier(n_neighbors=1)
knn_bal.fit(X_treino, y_treino)
y_teste_knn = knn_bal.predict(X_teste)

### Matriz de confusão

In [ ]:
plot_confusion_matrix(knn_bal, X_teste, y_teste, cmap='Blues')

In [ ]:
print(f'Acurácia:{accuracy_score(y_teste, y_teste_knn)}\n\
Precisão:{precision_score(y_teste, y_teste_knn)}\n\
Recall:{recall_score(y_teste, y_teste_knn)}')

### Curva ROC

In [ ]:
plot_roc_curve(knn_bal, X_teste, y_teste)

# Comparando todos os modelos

In [ ]:
classifiers = [rlog_1, knn, rlog_bal, knn_bal]
ax = plt.gca()
for i in classifiers:
    plot_roc_curve(i, X_teste, y_teste, ax=ax)

In [ ]:
df.shape
df.info()
df.sample(10)

In [ ]:
df.info()
df.describe().round(2)

In [ ]:
df.describe(include=np.object)

In [ ]:
plt.clf()
df.hist(figsize=(10,7))
plt.tight_layout()
plt.show

In [ ]:
# Histogramas "somados" stacked
fig = px.histogram(df,x='satisfaction_level',color="left")
fig.show()

In [ ]:
sns.countplot(x="satisfaction_level",hue="left", data=df)

In [ ]:
variaveis=list(df.columns)
target="left"
variaveis
target

In [ ]:
len(variaveis)//2

In [ ]:
# Histogramas separados overlay!!
#fig, ax = plt.subplots(15, 2, figsize=(10, 20))
linhas=(len(variaveis)//2)
if len(variaveis)%2:
    linhas=linhas+1
fig, axes = plt.subplots(linhas,2, figsize=(10, 20))
bins=10
ax=axes.ravel()
ficou = df[target] == 0
saiu = df[target] == 1
i=0
for var in variaveis:
    #a=df[ficou][var]
    #b=df[saiu][var]
    ax[i].hist(df[ficou][var],bins=bins, alpha=.5)
    ax[i].hist(df[saiu][var],bins=bins, alpha=.5)
    ax[i].set_title(var)
    #ax[i].yticks(())
    ax[i].set_xlabel("Feature magnitude")
    ax[i].set_ylabel("Frequency")
    ax[i].legend(["ficou", "saiu"], loc="best")
    i+=1
fig.tight_layout()

In [ ]:
correlação=df.corr()
correlação

In [ ]:
correlação.left.sort_values(ascending=False)

In [ ]:
df.columns
variaveis=list(df.columns)
target=["left"]
variaveis
target

In [ ]:
preditores=list(set(variaveis)-set(target))
variaveis=target+preditores
correlação=df[variaveis].corr()
correlação

In [ ]:
plt.rc('figure', figsize=(12, 8))
sns.heatmap(correlação,annot=True, vmin=-1, vmax=1)

In [ ]:
#sns.pairplot(df[variaveis], kind="scatter", hue=target[0])
sns.pairplot(df[variaveis], kind="scatter")

In [ ]:
sns.pairplot(df, y_vars=target, x_vars=preditores, hue=target[0])

## Fatores

In [ ]:
df.info()
unicos=df.nunique()
unicos

In [ ]:
# Tentar as que tem até 10
unicas=10
variaveis_categoricas=[]
for variavel in unicos.index:
    quant=unicos.loc[variavel]
    if quant <= unicas:
        variaveis_categoricas.append(variavel)
        print("Variavel {}, {} ocorrências únicas".format(variavel,quant))
        print(df[variavel].sort_values().unique())

In [ ]:
#left, sales, salary, promotion_last_5years, work_accident

variaveis_categoricas

In [ ]:
variaveis_categoricas.remove("number_project")
variaveis_categoricas.remove("time_spend_company")
variaveis_categoricas


In [ ]:
df[variaveis_categoricas].info()

In [ ]:
variaveis_dummy=variaveis_categoricas[0:3]
variaveis_dummy

In [ ]:
for variavel in variaveis_categoricas:
    df[variavel]=df[variavel].astype("category")
df.info()


In [ ]:
variaveis_numericas=list((df.columns))
variaveis_numericas=list(set(variaveis_numericas)-set(variaveis_categoricas))
variaveis_numericas

### Histogramas

In [ ]:
for variavel in variaveis_categoricas:
    df[variavel].value_counts().plot(kind="bar")
    plt.xlabel(variavel)
    plt.ylabel("Contagem")
    plt.title(variavel)
    plt.show()

### Boxplots - Identificando, verificando e tratando outliers


In [ ]:
for cat in variaveis_categoricas:
    for num in variaveis_numericas:
        sns.boxplot(x=cat, y=num, data=df)
        plt.xlabel(cat)
        plt.ylabel(num)
        plt.title(cat)
        plt.show()


In [ ]:
# com plotly
for cat in variaveis_categoricas:
    for num in variaveis_numericas:
        #sns.boxplot(x=cat, y=num, data=df)
        fig=px.box(df,y=num,x=cat)
        fig.show()


## Criação das variáveis dummies

In [ ]:
variaveis_categoricas.remove(target[0])
variaveis_dummy.remove(target[0])
variaveis_categoricas
variaveis_dummy

In [ ]:
# Trata variaveis binárias (0-1)
for variavel in variaveis_dummy:
    df=pd.concat([df, pd.get_dummies(df[variavel].astype(object), prefix=variavel, drop_first=True)], axis=1)
    df.drop(columns=[variavel], inplace=True)
df.sample(5)
df.columns

In [ ]:
#Trata variaveis categóricas
variaveis_categoricas=list(set(variaveis_categoricas)-set(variaveis_dummy))
for variavel in variaveis_categoricas:
    df=pd.concat([df, pd.get_dummies(df[variavel].astype(object), prefix=variavel)], axis=1)
    df.drop(columns=[variavel], inplace=True)
df.sample(5)
df.columns

In [ ]:
preditores=list(df.columns)
target
preditores=list(set(preditores)-set(target))
preditores
target

## Escala dos preditores

In [ ]:
def verifica_escala(preditores,df):
    x=df[preditores].values
    plt.plot(x.min(axis=0),"o", label="min")
    plt.plot(x.max(axis=0),"^", label="max")
    plt.legend(loc="best")
    plt.xlabel("Feature index")
    plt.ylabel("Feature magnitude")
    #plt.yscale("log")
    print(df[preditores].max().round(3))

In [ ]:
verifica_escala(preditores, df)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df[preditores].values)
x=scaler.transform(df[preditores].values)
df_scaled=pd.DataFrame(x,columns=preditores)
df_scaled[target]=df[target]
df_scaled.head()
df_original=df.copy()
df=df_scaled.copy()

## treino e teste


In [ ]:
df_treino, df_teste= train_test_split(df, test_size=0.3, random_state=42, stratify=df[target])

In [ ]:
df_treino.shape
df_teste.shape

# Regressão logística - statsmodel

In [ ]:
preditores=list(df.columns)
target
preditores=list(set(preditores)-set(target))
preditores
target

In [ ]:
#Não Balanceado
X_treino=df_treino[preditores].values
X_treino=sm.add_constant(X_treino)
y_treino=df_treino[target].values
#y_treino=y_treino.ravel()

X_teste=df_teste[preditores].values
X_teste=sm.add_constant(X_teste)
y_teste=df_teste[target].values


In [ ]:
#est = sm.Logit(y_train.ravel(), X_train).fit()
#Note that the first argument here is y, followed by x.
rlog_1= sm.Logit(y_treino, X_treino).fit()


In [ ]:
rlog_1.summary()
rlog_1.summary2()

In [ ]:
y_proba=rlog_1.predict(X_treino).round(3)
y_proba[1:10]

In [ ]:
# Predição treino
y_treino_pred=(rlog_1.predict(X_treino) >= 0.5).astype(int)
y_treino_pred[1:10]

In [ ]:
# Predição teste
y_teste_pred=(rlog_1.predict(X_teste) >= 0.5).astype(int)
y_teste_pred[1:10]

## Matriz de confusão - statsmodel

In [ ]:
matriz_conf=rlog_1.pred_table()
matriz_conf

In [ ]:
# Para statsmodels
#cm = confusion_matrix(y, model.predict(x))

def plot_matriz_confusão(cm):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(cm)
    ax.grid(False)
    ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 0s', 'Predicted 1s'))
    ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 0s', 'Actual 1s'))
    ax.set_ylim(1.5, -0.5)
    for i in range(2):
        for j in range(2):
            ax.text(j, i, cm[i, j], ha='center', va='center', color='red')
    plt.show()


In [ ]:
plot_matriz_confusão(matriz_conf)

In [ ]:
#Do sckit 
#- Acurácia: % de predições corretas
#- Precisão: % de predições corretas dentro da classe positiva
#- Recall: % de predições corretas dentro da classe esperada como positiva
print(classification_report(y_treino, y_treino_pred, target_names=['No', 'Yes']))

In [ ]:
print(classification_report(y_teste, y_teste_pred, target_names=['No', 'Yes']))

# Regressão logística - sklearn

In [ ]:
preditores
target

## Não balanceado

In [ ]:
# Não balanceado
X_treino=df_treino[preditores].values
y_treino=(df_treino[target].values).ravel()

X_treino.shape
y_treino.shape
X_teste=df_teste[preditores].values
y_teste=np.array(df_teste[target]).ravel()
y_treino[1:10]

In [ ]:
rlog_1 = LogisticRegression(solver = 'sag', max_iter = 10000)
rlog_1.fit(X_treino, y_treino)


In [ ]:
rlog_1.intercept_
rlog_1.coef_.round(3)

In [ ]:
y_proba= rlog_1.predict_proba(X_treino).round(3)
y_proba[1:10]

In [ ]:
y_treino_pred=rlog_1.predict(X_treino)
y_treino_pred[1:10]

In [ ]:
y_teste_rlog=rlog_1.predict(X_teste)
y_teste_rlog[1:10]

In [ ]:
rlog_1.score(X_treino, y_treino)

In [ ]:
rlog_1.score(X_teste, y_teste)

## Matriz de confusão - scklearn

In [ ]:
#confusion_matrix(y, model.predict(x))
confusion_matrix(y_treino,y_treino_pred)


In [ ]:
plot_confusion_matrix(rlog_1, X_treino, y_treino, cmap='Blues')

In [ ]:
plot_confusion_matrix(rlog_1, X_teste, y_teste, cmap='Blues')

In [ ]:
#Do sckit 
#- Acurácia: % de predições corretas
#- Precisão: % de predições corretas dentro da classe positiva
#- Recall: % de predições corretas dentro da classe esperada como positiva
print(classification_report(y_treino, y_treino_pred, target_names=['No', 'Yes']))

In [ ]:
#Do sckit 
#- Acurácia: % de predições corretas
#- Precisão: % de predições corretas dentro da classe positiva
#- Recall: % de predições corretas dentro da classe esperada como positiva
print(classification_report(y_teste, y_teste_rlog, target_names=['No', 'Yes']))
print(f'Acurácia:{accuracy_score(y_teste, y_teste_rlog)}\n\
Precisão:{precision_score(y_teste, y_teste_rlog)}\n\
Recall:{recall_score(y_teste, y_teste_rlog)}')


## Curva ROC e AUC

In [ ]:
plot_roc_curve(rlog_1, X_teste, y_teste)

In [ ]:
def classificação(modelo,target,preditores,treino,teste):
    x_treino=treino[preditores].values
    y_treino=treino[target].values
    y_treino=y_treino.ravel()
    
    x_teste=teste[preditores].values
    y_teste=teste[target].values
    
    modelo.fit(x_treino, y_treino)
    y_treino_pred=modelo.predict(x_treino)
    y_teste_pred=modelo.predict(x_teste)
    print(modelo)
    plot_confusion_matrix(modelo, x_teste,y_teste, cmap='Blues')
    print("Acurácia no treino: {:.3f}".format(modelo.score(x_treino, y_treino)))
    print("Acuracia no teste: {:.3f}".format(modelo.score(x_teste, y_teste)))
    print("Precisão no teste: {:.3f}".format(precision_score(y_teste, y_teste_pred)))
    print("Recall no teste: {:.3f}".format(recall_score(y_teste, y_teste_pred)))
    print("F1 no teste: {:.3f}".format(f1_score(y_teste, y_teste_pred)))
    plot_roc_curve(modelo, x_teste, y_teste)
    mod={}
    mod["modelo"]=modelo
    mod["acuracia_teste"]=modelo.score(x_teste, y_teste)
    mod["acuracia_treino"]=modelo.score(x_treino, y_treino)
    return mod

In [ ]:
lista_modelos=[]

In [ ]:

modelo=LogisticRegression(solver = 'sag', max_iter = 10000, random_state=42)

result=classificação(modelo,target, preditores, df_treino, df_teste)
modelo=result["modelo"]
lista_modelos.append(result)

## KNN - verificando o valor de K ideal

## Verificando o número e a acurácia

In [ ]:
x_treino=df_treino[preditores].values
y_treino=df_treino[target].values
y_treino=y_treino.ravel()
x_teste=df_teste[preditores].values
y_teste=df_teste[target].values

valores=range(1,10)

ac_treino=[]
ac_teste=[]
    
for val in valores:
    modelo= KNeighborsClassifier(n_neighbors=val)
    modelo.fit(x_treino, y_treino)
    #y_treino_pred=modelo.predict(x_treino)
    #y_teste_pred=modelo.predict(x_teste)
    ac_treino.append(modelo.score(x_treino, y_treino))
    ac_teste.append(modelo.score(x_teste, y_teste))
    
df_acuracia=pd.DataFrame({"Parametro":valores,"acuracia_treino":ac_treino,"acuracia_teste":ac_teste }, index=valores) 
df_acuracia[["acuracia_treino","acuracia_teste"]].plot()
df_acuracia

In [ ]:
modelo = KNeighborsClassifier(n_neighbors=1)

result=classificação(modelo,target, preditores, df_treino, df_teste)
modelo=result["modelo"]
lista_modelos.append(result)

## Árvore de Decisão 

## Verificando a profundidade da árvore e a acurácia

In [ ]:
x_treino=df_treino[preditores].values
y_treino=df_treino[target].values
y_treino=y_treino.ravel()
x_teste=df_teste[preditores].values
y_teste=df_teste[target].values

valores=range(2,15)

ac_treino=[]
ac_teste=[]
    
for val in valores:
    modelo= DecisionTreeClassifier(max_depth=val)
    modelo.fit(x_treino, y_treino)
    #y_treino_pred=modelo.predict(x_treino)
    #y_teste_pred=modelo.predict(x_teste)
    ac_treino.append(modelo.score(x_treino, y_treino))
    ac_teste.append(modelo.score(x_teste, y_teste))
    
df_acuracia=pd.DataFrame({"Parametro":valores,"acuracia_treino":ac_treino,"acuracia_teste":ac_teste }, index=valores) 
df_acuracia[["acuracia_treino","acuracia_teste"]].plot()
df_acuracia

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
modelo = DecisionTreeClassifier(max_depth=9)

result=classificação(modelo,target, preditores, df_treino, df_teste)
modelo=result["modelo"]
lista_modelos.append(result)

### Visualizando a árvore

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plot_tree(modelo, feature_names=preditores, filled=True,
          class_names=['Fica', 'Sai']);

In [ ]:
modelo.feature_importances_
preditores
importancia=pd.Series(modelo.feature_importances_, index=preditores)
importancia=importancia.sort_values(ascending=True)
importancia.plot(kind="barh")

## SVM

## Verificando o valor de C

In [ ]:
x_treino=df_treino[preditores].values
y_treino=df_treino[target].values
y_treino=y_treino.ravel()
x_teste=df_teste[preditores].values
y_teste=df_teste[target].values

valores=range(1,35)

ac_treino=[]
ac_teste=[]
    
for val in valores:
    modelo=  SVC(C=val)
    modelo.fit(x_treino, y_treino)
    #y_treino_pred=modelo.predict(x_treino)
    #y_teste_pred=modelo.predict(x_teste)
    ac_treino.append(modelo.score(x_treino, y_treino))
    ac_teste.append(modelo.score(x_teste, y_teste))
    
df_acuracia=pd.DataFrame({"Parametro":valores,"acuracia_treino":ac_treino,"acuracia_teste":ac_teste }, index=valores) 
df_acuracia[["acuracia_treino","acuracia_teste"]].plot()
df_acuracia

## Verificando o valor de gamma

In [ ]:
x_treino=df_treino[preditores].values
y_treino=df_treino[target].values
y_treino=y_treino.ravel()
x_teste=df_teste[preditores].values
y_teste=df_teste[target].values

valores=list(np.arange(0.1,5,0.5))

ac_treino=[]
ac_teste=[]
    
for val in valores:
    modelo=  SVC(C=30, gamma=val)
    modelo.fit(x_treino, y_treino)
    #y_treino_pred=modelo.predict(x_treino)
    #y_teste_pred=modelo.predict(x_teste)
    ac_treino.append(modelo.score(x_treino, y_treino))
    ac_teste.append(modelo.score(x_teste, y_teste))
    
df_acuracia=pd.DataFrame({"Parametro":valores,"acuracia_treino":ac_treino,"acuracia_teste":ac_teste }, index=valores) 
df_acuracia[["acuracia_treino","acuracia_teste"]].plot()
df_acuracia

In [ ]:

modelo = SVC(C=30, gamma=4.1)


result=classificação(modelo,target, preditores, df_treino, df_teste)
modelo=result["modelo"]
lista_modelos.append(result)


# Comparando todos os modelos

In [ ]:
lista_modelos

In [ ]:
def compara_modelos(lista,target,preditores, teste):
    x=df[preditores].values
    y=df[target].values
    ax = plt.gca()
    for i in lista:
        modelo=i["modelo"]
        plot_roc_curve(modelo, x, y, ax=ax)

In [ ]:
compara_modelos(lista_modelos, target, preditores, df_teste)

# Aplicando balanceamento

In [ ]:
preditores=list(df.columns)
target
preditores=list(set(preditores)-set(target))
preditores
target

In [ ]:
#from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_treino=df_treino[preditores].values
y_treino=(df_treino[target].values).ravel()
y_treino.shape
X_treino_bal, y_treino_bal = smote.fit_sample(X_treino, y_treino)
X_treino_bal.shape
y_treino_bal.shape
y_treino_bal=y_treino_bal.reshape((-1,1))
y_treino_bal.shape

variaveis=preditores+target

df_treino_bal=pd.DataFrame(X_treino_bal, columns=preditores)
df_treino_bal[target[0]]=y_treino_bal
df_treino_bal.sample(5)

In [ ]:
# balanceado
X_treino=df_treino_bal[preditores].values
y_treino=(df_treino_bal[target].values).ravel()
X_treino.shape
y_treino.shape
X_teste=df_teste[preditores].values
y_teste=np.array(df_teste[target]).ravel()
y_treino[1:10]

In [ ]:
rlog_bal = LogisticRegression(solver = 'sag', max_iter = 10000).fit(X_treino, y_treino)


In [ ]:
rlog_bal.intercept_
rlog_bal.coef_.round(3)

In [ ]:
y_proba= rlog_bal.predict_proba(X_treino).round(3)
y_proba[1:10]

In [ ]:
y_treino_pred=rlog_bal.predict(X_treino)
y_treino_pred[1:10]

In [ ]:
y_teste_rlog=rlog_bal.predict(X_teste)
y_teste_rlog[1:10]

In [ ]:
rlog_bal.score(X_treino, y_treino)

In [ ]:
rlog_bal.score(X_teste, y_teste)

## Matriz de confusão - scklearn

In [ ]:
#confusion_matrix(y, model.predict(x))
confusion_matrix(y_treino,y_treino_pred)


In [ ]:
plot_confusion_matrix(rlog_bal, X_treino, y_treino, cmap='Blues')

In [ ]:
plot_confusion_matrix(rlog_bal, X_teste, y_teste, cmap='Blues')

In [ ]:
#Do sckit 
#- Acurácia: % de predições corretas
#- Precisão: % de predições corretas dentro da classe positiva
#- Recall: % de predições corretas dentro da classe esperada como positiva
print(classification_report(y_treino, y_treino_pred, target_names=['No', 'Yes']))

In [ ]:
#Do sckit 
#- Acurácia: % de predições corretas
#- Precisão: % de predições corretas dentro da classe positiva
#- Recall: % de predições corretas dentro da classe esperada como positiva
print(classification_report(y_teste, y_teste_rlog, target_names=['No', 'Yes']))
print(f'Acurácia:{accuracy_score(y_teste, y_teste_rlog)}\n\
Precisão:{precision_score(y_teste, y_teste_rlog)}\n\
Recall:{recall_score(y_teste, y_teste_rlog)}')


## Curva ROC e AUC

In [ ]:
plot_roc_curve(rlog_bal, X_teste, y_teste)

## KNN

In [ ]:
knn_bal = KNeighborsClassifier(n_neighbors=1)
knn_bal.fit(X_treino, y_treino)
y_teste_knn = knn_bal.predict(X_teste)

### Matriz de confusão

In [ ]:
plot_confusion_matrix(knn_bal, X_teste, y_teste, cmap='Blues')

In [ ]:
print(f'Acurácia:{accuracy_score(y_teste, y_teste_knn)}\n\
Precisão:{precision_score(y_teste, y_teste_knn)}\n\
Recall:{recall_score(y_teste, y_teste_knn)}')

### Curva ROC

In [ ]:
plot_roc_curve(knn_bal, X_teste, y_teste)

# Comparando todos os modelos

In [ ]:
classifiers = [rlog_1, knn, rlog_bal, knn_bal]
ax = plt.gca()
for i in classifiers:
    plot_roc_curve(i, X_teste, y_teste, ax=ax)

In [ ]:
df.info()
df.describe().round(2)

In [ ]:
#df.fillna(0, inplace=True)
#df.info()

In [ ]:
df.describe(include=np.object)

In [ ]:
df.info()
unicos=df.nunique()
unicos

## Defindo variaveis categóricas numericas

In [ ]:
def analisa_variaveis(df,variaveis,nunicas):
    lista=[]
    unicos=df[variaveis].nunique()
    for variavel in unicos.index:
        quant=unicos.loc[variavel]
        if quant <= nunicas:
            lista.append(variavel)
            print("Variavel {}, {} ocorrências únicas".format(variavel,quant))
            print(df[variavel].sort_values().unique())
    return lista

In [ ]:
variaveis_numericas= df.select_dtypes(include=[np.number]).columns
variaveis_categoricas= df.select_dtypes(include="category").columns
df[variaveis_numericas].info()
df[variaveis_categoricas].info()

In [ ]:
variaveis_objeto=df.select_dtypes(include="object").columns
df[variaveis_objeto].info()

### Histogramas

In [ ]:
for variavel in variaveis_categoricas:
    df[variavel].value_counts().plot(kind="bar")
    plt.xlabel(variavel)
    plt.ylabel("Contagem")
    plt.title(variavel)
    plt.show()

### Boxplots - Identificando, verificando e tratando outliers


In [ ]:
for cat in variaveis_categoricas:
    for num in variaveis_numericas:
        sns.boxplot(x=cat, y=num, data=df)
        plt.xlabel(cat)
        plt.ylabel(num)
        plt.title(cat)
        plt.show()


In [ ]:
df.info()

In [ ]:
variaveis=list(variaveis_numericas)+list(variaveis_categoricas)
variaveis.remove("cluster")
preditores=variaveis
target=["cluster"]
variaveis=["cluster"]+variaveis
variaveis

In [ ]:

df["cluster"]=df["cluster"].astype("int")
df["cluster"]=df["cluster"].astype("int")
df["cluster"]=df["cluster"].astype("int")
df["cluster"]=df["cluster"].astype("int")
df["cluster"]=df["cluster"].astype("int")

correlação=df[variaveis].corr()
correlação
plt.rc('figure', figsize=(12, 8))
sns.heatmap(correlação,annot=True, vmin=-1, vmax=1)

In [ ]:
sns.pairplot(df[variaveis], kind="scatter")

In [ ]:
sns.pairplot(df, y_vars=target, x_vars=preditores, hue=target[0])

In [ ]:
# adjusted rand score is 1, as the clustering is exactly the same
# df_kmeans, df_mean_s, df_dbscan, df_GMM, df_HC, df_ch

In [ ]:
from sklearn.metrics import adjusted_rand_score
target=df_HC
lista=[df_kmeans,df_mean_s,df_dbscan,df_GMM,df_HC,df_ch]

for df in lista:
    print("ARI: {:.2f}".format(adjusted_rand_score(target["cluster"].values, df["cluster"].values)))


## EDA com K-mean

In [ ]:
df=df_HC.copy()

## EDA Básico

In [ ]:
df.info()
df.describe().round(2)

In [ ]:
#df.fillna(0, inplace=True)
#df.info()

In [ ]:
df.describe(include=np.object)

In [ ]:
df.info()
unicos=df.nunique()
unicos

## Defindo variaveis categóricas numericas

In [ ]:
def analisa_variaveis(df,variaveis,nunicas):
    lista=[]
    unicos=df[variaveis].nunique()
    for variavel in unicos.index:
        quant=unicos.loc[variavel]
        if quant <= nunicas:
            lista.append(variavel)
            print("Variavel {}, {} ocorrências únicas".format(variavel,quant))
            print(df[variavel].sort_values().unique())
    return lista

In [ ]:
variaveis_numericas= df.select_dtypes(include=[np.number]).columns
variaveis_categoricas= df.select_dtypes(include="category").columns
df[variaveis_numericas].info()
df[variaveis_categoricas].info()

In [ ]:
variaveis_objeto=df.select_dtypes(include="object").columns
df[variaveis_objeto].info()

### Histogramas

In [ ]:
for variavel in variaveis_categoricas:
    df[variavel].value_counts().plot(kind="bar")
    plt.xlabel(variavel)
    plt.ylabel("Contagem")
    plt.title(variavel)
    plt.show()

### Boxplots - Identificando, verificando e tratando outliers


In [ ]:
for cat in variaveis_categoricas:
    for num in variaveis_numericas:
        sns.boxplot(x=cat, y=num, data=df)
        plt.xlabel(cat)
        plt.ylabel(num)
        plt.title(cat)
        plt.show()


In [ ]:
df.info()

In [ ]:
variaveis=list(variaveis_numericas)+list(variaveis_categoricas)
variaveis.remove("cluster")
preditores=variaveis
target=["cluster"]
variaveis=["cluster"]+variaveis
variaveis

In [ ]:

df["cluster"]=df["cluster"].astype("int")
df["cluster"]=df["cluster"].astype("int")
df["cluster"]=df["cluster"].astype("int")
df["cluster"]=df["cluster"].astype("int")
df["cluster"]=df["cluster"].astype("int")

correlação=df[variaveis].corr()
correlação
plt.rc('figure', figsize=(12, 8))
sns.heatmap(correlação,annot=True, vmin=-1, vmax=1)

In [ ]:
sns.pairplot(df[variaveis], kind="scatter")

In [ ]:
sns.pairplot(df, y_vars=target, x_vars=preditores, hue=target[0])